In [ ]:
import pandas as pd
import numpy as np

filename_timestamp = '2019071710'
file_conn = 'input\Circuit Trace IMC3 IMC4 20190712.csv';
file_event = 'input\Event IMC3 PDU offline 20190617.csv'
file_pdu = 'input\Export Rack PDU IMC3 IMC4 ' + filename_timestamp + '.csv'
file_piq_pdu = 'input\PIQ PDU ' + filename_timestamp + '.csv'

df_event = pd.read_csv(file_event);
df_conn = pd.read_csv(file_conn);
df_pdu = pd.read_csv(file_pdu, comment='#', header=None);
df_piq_pdu = pd.read_csv(file_piq_pdu, comment='#');

In [ ]:
# name is col 7
# ip address is col 38
# proxy index is col 40
# serial number is col4

bool_make_is_raritan = df_pdu[2].str.match('Raritan', na=False)
bool_status_is_archive = df_pdu[15].str.match('Archived', na=False)
df_pdu2 = df_pdu.loc[bool_make_is_raritan & ~bool_status_is_archive, [4, 7,38, 40]]
df_pdu2['Name'] = df_pdu2[7].str.extract('([34][ABSW][0-9]+[RB][0-9])', expand=True)
df_pdu2['Proxy'] = df_pdu2[40].fillna(0).astype(int)
df_pdu2['IP address'] = df_pdu2[38]
df_pdu2['Serial Number'] = df_pdu2[4]
df_pdu2['Serial Number'] = df_pdu2[4]
df_pdu2['IP with Proxy'] = df_pdu2['IP address'] + ' - ' + df_pdu2['Proxy'].astype(str)


#Fix name that is not following convention of 3A45R1
bool_name_is_null = df_pdu2['Name'].isnull()
df_pdu2.loc[bool_name_is_null, 'Name'] = df_pdu2.loc[bool_name_is_null, 7]

In [ ]:
df_pdu3 = df_pdu2.drop([4, 7, 38, 40], axis=1)

#List of PDU with names not following convention
print("Total Raritan Rack PDU in IMC3:", df_pdu3.shape[0])
print("PDU with non-conventional names:", df_pdu3.loc[bool_name_is_null].shape[0])
display(df_pdu3.loc[bool_name_is_null,:])

In [ ]:
bool_make_is_raritan_piq = df_piq_pdu['Manufacturer'].str.match('Raritan', na=False)

print("Total Num of Raritan device:", df_piq_pdu.loc[bool_make_is_raritan_piq, :].shape[0], "\n")
print("Num by Raritan Model: \n", df_piq_pdu.loc[bool_make_is_raritan_piq, :].groupby('Model')['Label'].nunique(), "\n")

bool_model_is_PX3 = df_piq_pdu['Model'].str.match('PX3-', na=False)
df_piq_pdu2 = df_piq_pdu.loc[bool_make_is_raritan_piq & bool_model_is_PX3, ['Label', 'Serial Number', 'Name']]
print("Total Num of Raritan device:", df_piq_pdu2.shape[0], "\n")

#print(df_piq_pdu2)

#print(df_pdu3.loc[:,['IP with Proxy', 'Serial Number', 'Name']])

print("Num of dcTrack PDU:", df_pdu3.shape[0])


In [ ]:
print(df_pdu3.columns)
print(df_piq_pdu2.columns)

In [ ]:
#Match by Name
m = {'left_only': 'dcTrack', 'right_only': 'PIQ', 'both': 'dcTrack, PIQ'}
result = df_pdu3.merge(df_piq_pdu2, on=['Name'], how='outer', indicator='Scope')
result['Scope'] = result['Scope'].map(m)

result['Same IP Proxy'] = np.where(result['IP with Proxy'] == result['Label'], True, False)
bool_same_ipproxy = result['Same IP Proxy'] == True;
result = result.rename(index=str, columns={'IP with Proxy': 'dcTrack IP Proxy', 'Label': 'PIQ IP Proxy'})
print(result)
result.to_csv('output\pdu_name_diff_ipproxy_' + filename_timestamp + '.csv', columns=['Scope', 'Same IP Proxy', 'dcTrack IP Proxy', 'Name', 'PIQ IP Proxy'])

In [ ]:
#Match by IP Proxy

result2 = df_pdu3.merge(df_piq_pdu2, left_on=['IP with Proxy'], right_on=['Label'], how='outer', indicator='Scope')
result2['Scope'] = result2['Scope'].map(m)

result2['Same Name'] = np.where(result2['Name_x'] == result2['Name_y'], True, False)
bool_same_name = result2['Same Name'] == True
display(result2.loc[~bool_same_name, ['Scope','Name_x', 'IP with Proxy', 'Label', 'Name_y']])
result2 = result2.rename(index=str, columns={'Name_x': 'dcTrack Name', 'Name_y': 'PIQ Name', 'IP with Proxy': 'dcTrack IP Proxy', 'Label': 'PIQ IP Proxy'})
result2.to_csv('output\ipproxy_with_different_name_' + filename_timestamp + '.csv', columns=['Scope', 'Same Name', 'dcTrack Name', 'dcTrack IP Proxy', 'PIQ IP Proxy', 'PIQ Name'])

In [ ]:
#Trace circuit from Event log

In [ ]:
#Split series of circuit trace into df
df_circuit = df_conn['Circuit Trace'];
bool_circuit_match_pdu = df_circuit.str.match('[34][ABSW][0-9]+[RB][0-9]', na=True)
df_circuit2 = df_circuit.loc[bool_circuit_match_pdu].str.split(' - ', expand=True)

#Clean up df_circuit2 into #Rack PDU (3A44B1), Power Outlet (PDU-1F-C02-13L3:B1), PDU Panel Port (C02:13L3), 
#  PDU Panel (C02), Floor PDU (PDU-1F-C02), UPS Bank (UPS BANK B)
df_circuit3 = df_circuit2.copy()
df_circuit3['Rack PDU'] = df_circuit2.loc[:,0].str.extract('([34][ABSW][0-9]+[RB][0-9])', expand=True)
df_circuit3['Power Outlet'] = df_circuit2.loc[:,0].str.extract('(PDU-[0-9]+F-[A-Z][0-9]+-[0-9]+L[0-9]:[A-Z][0-9])', expand=True)
df_circuit3['PDU Panel Port'] = df_circuit2.loc[:,1]
df_circuit3['Power Panel'] = df_circuit2.loc[:,2].str.extract('([A-Z][0-9]+):', expand=True)
df_circuit3['Floor PDU'] = df_circuit2.loc[:,3].str.extract('(PDU-[0-9]F-[A-Z][0-9]+):', expand=True)
df_circuit3['UPS Bank'] = df_circuit2.loc[:,4].str.extract('(UPS BANK [A-Z]):', expand=True)

In [ ]:
df_circuit4 = df_circuit3.drop([0, 1, 2, 3, 4], axis=1)

print("Number of Rack PDU in Circuit:", df_circuit4.shape[0])
display(df_circuit4)

In [ ]:
#NaN in Power Outlet?

In [ ]:
#The following list of pdu is not in circuit trace
#Looks like DCO created their own floor PDU incorrectly
bool_pdu_isin_circuit = df_pdu3['Name'].isin(df_circuit4['Rack PDU'])

print("Number of Rack PDU not in Circuit Trace:", df_pdu3.loc[~bool_pdu_isin_circuit,:].shape[0], "\n")
print(df_pdu3.loc[~bool_pdu_isin_circuit,:])

df_pdu3.loc[~bool_pdu_isin_circuit,:].to_csv('output\pdu_does_not_have_circuit_' + filename_timestamp + '.csv', ",", index=False);

In [ ]:
#Add ip with proxy to df_circuit4
#df.join(other.set_index('key'), on='key')

df_circuit5 = df_circuit4.join(df_pdu3.set_index('Name'), on='Rack PDU')
print("Number of circuits with null values:", df_circuit5.isnull().any(axis=1).sum())
display(df_circuit5[df_circuit5.isnull().any(axis=1)])

In [ ]:
df_circuit5.to_csv('output\circuit_with_ip_proxy_' + filename_timestamp + '.csv')

In [ ]:
#duplicateRowsDF = dfObj[dfObj.duplicated(['Name'])]
#df.sort_values(by=['col1'])

bool_ipproxy_dup = df_pdu3.duplicated(['IP with Proxy'], keep=False)
print("Number of PDU with IP Proxy conflicts:", df_pdu3.loc[bool_ipproxy_dup,:].shape[0])
display(df_pdu3.loc[bool_ipproxy_dup,:].sort_values(by=['IP with Proxy']))

In [ ]:
print(df_event.columns)
display(df_event)

In [ ]:
#df.join(other.set_index('key'), on='key')

df_event2 = df_event.join(df_circuit5.set_index('IP with Proxy'), on='Target')
display(df_event2)


In [ ]:
#df.groupby('domain')['ID'].nunique()
print(df_event2.loc[:, ['Rack PDU', 'Target', 'Floor PDU', 'UPS Bank']].groupby('UPS Bank')['Rack PDU'].nunique())
print(df_event2.loc[:, ['Rack PDU', 'Target', 'Floor PDU', 'UPS Bank']].groupby('Floor PDU')['Rack PDU'].nunique())

In [ ]:
df_event2.loc[df_event2['Target'] == '10.218.76.151 - 8', :]